In [1]:
import requests
import pandas as pd
import json

In [2]:
def get_req(url):
    resp = requests.get(url)
    return json.loads(resp.text)

In [3]:
def urlify(s):
    return s.replace(" ", "%20")

In [4]:
# Read in local db files
df_cs = pd.read_csv("class_specs.csv")
df_cs.head()
df_affixes = pd.read_csv("affixes.csv")
df_affixes.head()
df_dungeons = pd.read_csv("dungeons.csv")
df_dungeons.head()

,id,name,timer
0,AD,Atal'dazar,1800
1,FH,Freehold,2160
2,KR,Kings' Rest,2340
3,ML,The MOTHERLODE!!,2340
4,SB,Siege of Boralus,2160


In [5]:
def json_to_entries(js):
    entries = []
    for run in js["rankings"]:
        run = run["run"]

        # Extract data from json
        du_name = run["dungeon"]["name"]
        di_level = run["mythic_level"]
        affixes_json = run["weekly_modifiers"]
        affixes = [j["name"] for j in affixes_json]
        class_specs = [(c["character"]["class"]["name"], c["character"]["spec"]["name"]) for c in run["roster"]]


        # "Atal'dazar" -> "AD"
        du_id = df_dungeons[df_dungeons.name==du_name]["id"].values[0]

        # ['Tyrannical', 'Teeming', 'Volcanic', 'Infested'] -> "TY_TE_VO_IN"
        try:
            affix_id = df_affixes[df_affixes.level_2==affixes[0]][df_affixes.level_4==affixes[1]][df_affixes.level_7==affixes[2]][df_affixes.level_10==affixes[3]]["id"].values[0]
        except IndexError:
            print(affixes)

        # ('Rogue', 'Subtlety') -> "RO_SU"
        cs_ids = []
        for cls, spec in class_specs:
            try:
                cs_ids.append(df_cs[df_cs.base_class==cls][df_cs.specialisation==spec]["id"].values[0])
            except IndexError:
                print(cls, spec)
                continue

        # Convert to single entries
        for cs_id in cs_ids:
            entries.append((cs_id, du_id, di_level, affix_id))

    return entries

In [6]:
regions = ["eu", "us", "tw", "kr"]
dungeons = df_dungeons.name.values
affixes_2 = ["Tyrannical", "Fortified"]
url_format = "https://raider.io/api/v1/mythic-plus/runs?season=season-bfa-1&region={}&dungeon={}&affixes={}"
urls = [url_format.format(r, urlify(d), a) for r in regions for d in dungeons for a in affixes_2]

In [7]:
# Get entries for all regions, dungeons and affixes**
entries = []
for url in urls:
    resp_json = get_req(url)
    if "error" in resp_json:
        print(resp_json)
        continue
    
    entries += json_to_entries(resp_json)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [8]:
# Convert to dataframe
df_entries = pd.DataFrame.from_records(entries, columns=["class_id", "dungeon_id", "difficulty_id", "affix_id"])
df_entries.head()

,class_id,dungeon_id,difficulty_id,affix_id
0,RO_SU,AD,19,TY_TE_VO_IN
1,HU_BM,AD,19,TY_TE_VO_IN
2,DK_BL,AD,19,TY_TE_VO_IN
3,MO_MW,AD,19,TY_TE_VO_IN
4,DH_HA,AD,19,TY_TE_VO_IN


In [9]:
# Aggregate rows into num_completed
cols = df_entries.columns.tolist()
agg_entries = df_entries.groupby(cols,as_index=False).size()
agg_entries = agg_entries.reset_index().rename(columns={0:'num_completed'})
agg_entries

,class_id,dungeon_id,difficulty_id,affix_id,num_completed
0,DH_HA,AD,15,FO_BO_SK_IN,4
1,DH_HA,AD,15,FO_SA_GR_IN,1
2,DH_HA,AD,15,TY_BU_SK_IN,1
3,DH_HA,AD,15,TY_RA_NE_IN,2
4,DH_HA,AD,16,FO_BO_SK_IN,6
5,DH_HA,AD,16,FO_SA_GR_IN,3
6,DH_HA,AD,16,TY_RA_NE_IN,2
7,DH_HA,AD,16,TY_TE_VO_IN,4
8,DH_HA,AD,17,FO_BO_SK_IN,8
9,DH_HA,AD,17,FO_SA_GR_IN,9


In [10]:
agg_entries.to_csv("ca270_entries.csv", header=True, index=False)